In [1]:
'''
Homework 3
--------------
Four parts to this problem:
	a) Reading a stream of images from a webcamera or folder, and displaying the video
	b) Threshold based detection
	c) Background differencing
	d) Visualizing motion history
--------------
'''
import cv2
import sys
import numpy as np #for math operations

In [2]:
a = np.random.random((5,5)) #basic debugging tool in math is still to print them and check manually!
print(a)

[[0.51666947 0.28670129 0.00449725 0.90841829 0.21968341]
 [0.09506912 0.15252036 0.58526796 0.01448878 0.11890857]
 [0.49968806 0.39806234 0.75216418 0.5917233  0.87156117]
 [0.91705995 0.65408826 0.59840855 0.37804321 0.23061293]
 [0.06692687 0.7887399  0.74825837 0.9437075  0.32811534]]


In [3]:
mask = a > 0.5 #See how logical operations can be used to thresholding?
print(mask.all() or mask.all())

False


In [4]:
#Fill in code in these definitions
def my_threshold_detect(src):
    '''
    Function that detects whether a pixel belongs to the foreground on RGB values
    Args: 
        src The source color image
    Returns: 
        dst The destination grayscale image where foreground pixels are colored white and the rest are colored black
    
    '''
    gray_image = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    # ret, dst = cv2.threshold(gray_image, 210, 255, cv2.THRESH_BINARY)
    # selecting the pre-determined threshold = 210
    dst = (gray_image > 210).astype(np.uint8)*255
    return dst
    pass

def my_frame_differencing(prev, curr):
    '''
    Function that does frame differencing between the current frame and the previous frame
    Args:
        src The current color image
        prev The previous color image
    Returns:
        dst The destination grayscale image where pixels are colored white if the corresponding pixel intensities in the current
    and previous image are not the same
    '''
    gray_curr = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY)
    gray_prev = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
    difference = cv2.absdiff(gray_curr, gray_prev)
    # selecting the pre-determined threshold = 40
    dst = (difference > 20).astype(np.uint8)*255
    return dst
    pass

def my_frame_differencing_adaptive(prev, curr):
    gray_curr = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY)
    gray_prev = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
    difference = cv2.absdiff(gray_curr, gray_prev)
    ret, dst = cv2.threshold(difference, 200, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    return dst
    pass

def my_frame_differencing_mean(prev_frames, curr):
    background = prev_frames[0]
    for i in range(len(prev_frames)):
        if i == 0:
            pass
        else:
            alpha = 1.0/(i + 1)
            beta = 1.0 - alpha
            background = cv2.addWeighted(prev_frames[i], alpha, background, beta, 0.0) 
    gray_curr = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY)
    gray_prev = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
    difference = cv2.absdiff(gray_curr, gray_prev)
    dst = (difference > 40).astype(np.uint8)*255
    return dst
    pass

def my_motion_energy(mh):
    '''
    Function that accumulates the frame differences for a certain number of pairs of frames
    Args:
        mh Vector of frame difference images
    Returns:
        dst The destination grayscale image to store the accumulation of the frame difference images
    '''
    mh1 = cv2.add(mh[0], mh[1])
    return cv2.add(mh[0], mh1)
    pass

In [7]:
# ----------------
# a) Reading a stream of images from a webcamera, and displaying the video
# ----------------
# For more information on reading and writing video: http://docs.opencv.org/modules/highgui/doc/reading_and_writing_images_and_video.html
# open the video camera no. 0
# Fill in code to read the given video file. 
cap = cv2.VideoCapture("railway2.avi")



# if not successful, exit program
if not cap.isOpened():
    print("Cannot open the video cam or file")
    sys.exit()

# create a window called "MyVideo0"
# cv2.namedWindow("MyVideo0", cv2.WINDOW_AUTOSIZE)

# read a new frame from video
ret, frame0 = cap.read()
if not ret:
    print("Cannot read a frame from video stream")

# show the frame in "Webcam" window
#cv2.imshow("Webcam", frame0)
w = int(cap.get(3))
h = int(cap.get(4))
out_1 = cv2.VideoWriter('threshold_tracker.avi', 0, 30, (w,h),0)
out_2 = cv2.VideoWriter('framediff_nonadaptive.avi', 0, 30, (w,h),0)
out_3 = cv2.VideoWriter('framediff_adaptive.avi', 0, 30, (w,h),0)
out_4 = cv2.VideoWriter('framediff_mean.avi', 0, 30, (w,h),0)
out_5 = cv2.VideoWriter('motionhistory.avi', 0, 30, (w,h),0)
out_6 = cv2.VideoWriter('Compare.avi', 0, 30, (w,h), 0)

# create windows
# cv2.namedWindow("MyVideo", cv2.WINDOW_AUTOSIZE)
# cv2.namedWindow("FrameDiff", cv2.WINDOW_AUTOSIZE)
# cv2.namedWindow("MotionEnergy", cv2.WINDOW_AUTOSIZE)

my_motion_history = []

mean_frames = list()
while(1): #playing the video
    # read a new frame from video
    ret, frame = cap.read()
    # if not successful, break loop
    if not ret:
        print("Cannot read a frame from video stream")
        break

    #cv2.imshow("Webcam", frame)

    # ----------------
    # b) Threshold based detection
    # ----------------
    frame_dst1 = my_threshold_detect(frame)
    out_1.write(frame_dst1)
    #cv2.imshow("Threshold", frame_dst1)

    # ----------------
    # c) Background differencing
    # ----------------

    # call my_frame_differencing - Non Adaptive function
    frame_dst2 = my_frame_differencing(frame0, frame)
    out_2.write(frame_dst2)
    #cv2.imshow("FrameDiff Non-Adaptive", frame_dst2)
    
    # call my_frame_differencing - Adaptive function
    frame_dst3 = my_frame_differencing_adaptive(frame0, frame)
    out_3.write(frame_dst3)
    #cv2.imshow("FrameDiff Adaptive", frame_dst3)
    frame_dst4 = frame_dst3
    # call my_frame_differencing - mean function
    mean_frames.append(frame)
    if len(mean_frames) > 5:
        frame_dst4 = my_frame_differencing_mean(mean_frames, frame)
        mean_frames.pop(0)
        out_4.write(frame_dst4)
        #cv2.imshow("FrameDiff Mean", frame_dst4)
        
    # Fill in code to update motion history
     
    # ----------------
    #  d) Visualizing motion history
    # ----------------
    
    # Add some frames initially to create motion history, here I used 3.
    frame_dstmh = my_frame_differencing(frame0, frame)
    myMH = frame_dstmh
    my_motion_history.append(frame_dstmh)
    if len(my_motion_history) > 4:
        # call my_motion_energy function
        myMH = my_motion_energy(my_motion_history)
        #cv2.imshow("MotionEnergy", myMH)
        my_motion_history.pop(0)
    out_5.write(myMH)
    
    # Quantative 
    cv2.putText(frame_dst1, 'Threshold Tracker', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_4)
    cv2.putText(frame_dst2, 'Non Adaptive', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_4)
    cv2.putText(frame_dst3, 'Adaptive', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_4)
    cv2.putText(frame_dst4, 'Mean', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_4)

    multi_frame = np.hstack((np.vstack((frame_dst1, frame_dst2)), np.vstack((frame_dst3, frame_dst4))))
    cv2.imshow("Quantative", multi_frame)
    out_6.write(multi_frame)
    frame0 = frame
    
    # wait for 'esc' key press for 30ms. If 'esc' key is pressed, break loop
    if cv2.waitKey(30) == 27:
        print("esc key is pressed by user")
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

Cannot read a frame from video stream
